# L1 Exercise 1: Creating a Table with PostgreSQL

### Walk through the basics of PostgreSQL. You will need to complete the following tasks:<li> Create a table 

in PostgreSQL, <li> Insert rows of data <li> Run a simple SQL query to validate the information. <br>
`#####` denotes where the code needs to be completed. 
    
Note: __Do not__ click the blue Preview button in the lower task bar

#### Import the library 
*Note:* An error might popup after this command has executed. If it does, read it carefully before ignoring. 

<center><h1><span style='color:blue'>Environment preparation</span></h1></center>

Udacity environment has been prepared to ease student task, i.e. has a Postgres instance available for training exercises.

Let's create one based on Kubernetes.

* Add Pyscopg2 module to Python
* Load in K8s Postgresql

In [1]:
# Load package
!pip install psycopg2-binary

<h3><span style='color:blue'>Using K8S PostgreSQL</span></h3>

Obviously you need a k8s avaible like: Minikube, Minishift, Docker (with K8s)

Helm is need to, go to [helm.sh](http://helm.sh)

In [2]:
from time import sleep
import os

In [3]:
helm_version = !helm version --short
assert helm_version[0][:2] == 'v3', "Expected HELM version not available, visit https://helm.sh"

#!curl -fsSL -o /tmp/get_helm.sh https://raw.githubusercontent.com/helm/helm/master/scripts/get-helm-3
#!chmod 700 /tmp/get_helm.sh
#!ls -al /tmp/
#!./get_helm.sh

In [4]:
!helm repo add bitnami https://charts.bitnami.com/bitnami

"bitnami" has been added to your repositories


In [5]:
CHART_INSTANCE_NAME = 'dend-l1e1'
os.environ['postgresql_port_instance_name'] = CHART_INSTANCE_NAME + "-postgresql"
os.getenv('postgresql_port_instance_name')

'dend-l1e1-postgresql'

In [6]:
helm_chart_out = !helm install {CHART_INSTANCE_NAME} stable/postgresql

In [7]:
postgresql_port_forward_command = helm_chart_out[-2].strip()
os.environ['postgresql_port_forward_command'] = postgresql_port_forward_command
os.getenv('postgresql_port_forward_command')

'kubectl port-forward --namespace default svc/dend-l1e1-postgresql 5432:5432 &'

In [9]:
# Waits until postgresl is running on 
max_checks_postgresql_run = 20

!kubectl get pods

while max_checks_postgresql_run > 0:

    postgres_is_running = !kubectl get pods|fgrep {CHART_INSTANCE_NAME}|fgrep "1/1"|fgrep "Running"
    
    if len(postgres_is_running) > 0 and not postgres_is_running[0] == 'No resources found.':
        break
    else:
        sleep(5)

        max_checks_postgresql_run -= 1

!kubectl get pods
assert max_checks_postgresql_run > 0, "Probably Postgresql is not running"

NAME                     READY   STATUS    RESTARTS   AGE
dend-l1e1-postgresql-0   0/1     Running   0          35s
dend-l1e2-cassandra-0    1/1     Running   0          49m
NAME                     READY   STATUS    RESTARTS   AGE
dend-l1e1-postgresql-0   1/1     Running   0          40s
dend-l1e2-cassandra-0    1/1     Running   0          49m


<h3><span style='color:blue'>Open Proxy to PostgreSQL on K8s</span></h3>
Run next command in a separate terminal (if not run on Jupyter ;-))

In [10]:
%%script env postgres_port_forward_command="$postgres_port_forward_command" --bg bash --out console_out
nohup kubectl port-forward --namespace default svc/dend-l1e1-postgresql 5432:5432 &


In [11]:
# Getting postgresql password from console out
postgresql_password = helm_chart_out[15].split('(')[1][:-1]
postgresql_password = !{postgresql_password}
postgresql_password = postgresql_password[0]

In [12]:
# Getting console command to connect with current instance of postgres
k8s_psql_command = helm_chart_out[19].strip().replace("$POSTGRES_PASSWORD", postgresql_password) + " -c "

<h3><span style='color:blue'>Check Postgresql availibity</span></h3>
We had created an Postgresql on a K8s infraestructure, next we will test if it is avaiable

In [13]:
# Checks postgresql connection
select_1_postgresql_out = !{k8s_psql_command} 'SELECT 1;'
assert len(select_1_postgresql_out) > 0, 'Postgresql -select 1- failed, check it'
!{k8s_psql_command} 'SELECT version();'

                                                 version                        
                         
--------------------------------------------------------------------------------
-------------------------
 PostgreSQL 11.6 on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1
) 6.3.0 20170516, 64-bit
(1 row)

pod "dend-l1e1-postgresql-client" deleted


<h3><span style='color:blue'>Initialize Postgresql Student DB for excercise</span></h3>

In [14]:
!{k8s_psql_command} "CREATE ROLE student WITH LOGIN ENCRYPTED PASSWORD 'student'"

CREATE ROLE
pod "dend-l1e1-postgresql-client" deleted


In [15]:
!{k8s_psql_command} 'alter user student createdb;'

ALTER ROLE
pod "dend-l1e1-postgresql-client" deleted


In [16]:
!{k8s_psql_command} 'create database studentdb;'

If you don't see a command prompt, try pressing enter.
CREATE DATABASE
pod "dend-l1e1-postgresql-client" deleted


In [17]:
!{k8s_psql_command} 'grant all privileges on database studentdb to student;'

GRANT
pod "dend-l1e1-postgresql-client" deleted


In [18]:
!{k8s_psql_command} 'SELECT usename, usecreatedb FROM pg_user;'

 usename  | usecreatedb 
----------+-------------
 postgres | t
 student  | t
(2 rows)

pod "dend-l1e1-postgresql-client" deleted


<center><h1><span style='color:blue'>Udacity - DEND - L1E1 - Exercise</span></h1></center>

### _Create a connection to the database_

In [20]:
import psycopg2

In [21]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 port=5432 dbname=studentdb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### _Next use that connect to get a cursor that can be used to execute queries._

In [22]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

### _Set automatic commit to be true so that each action is committed without having to call conn.commit() after each command._ 

In [23]:
conn.set_session(autocommit=True)

## _Let's Test our Connection and our Error Handling_

_We are trying to do a select * on a table we have not created yet. We should expect to see a nicely handled error._

In [24]:
try: 
    cur.execute("select * from songs")
except psycopg2.Error as e:
    print(e)

relation "songs" does not exist
LINE 1: select * from songs
                      ^



### Let's create a database to do the work in. 

In [25]:
## TO-DO: Add the database name within the CREATE DATABASE statement. You can choose your own db name.
try: 
    cur.execute("create database music")
except psycopg2.Error as e:
    print(e)

#### TO-DO: Add the database name in the connect statement. Let's close our connection to the default database, reconnect to the Udacity database, and get a new cursor.

In [26]:
## TO-DO: Add the database name within the connect statement
flag_error_reconnect = False

try: 
    cur.close()
    conn.close()    
except psycopg2.Error as e:
    flag_error_reconnect = True
    print("Error: Could not get curser to the Database")
    print(e)

if not flag_error_reconnect:
    try:
        conn = psycopg2.connect("host=127.0.0.1 port=5432 dbname=music user=student password=student")
    except psycopg2.Error as e:
        flag_error_reconnect = True
        print("Error: Could not connect to Database")
        print(e)
        
if not flag_error_reconnect:
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        flag_error_reconnect = True
        print("Error: Could not get curser to the Database")
        print(e)
        
if not flag_error_reconnect:
    conn.set_session(autocommit=True)

### _Let's image we would like to start creating a song Library that contains a list of songs, including the song name, artist name, year, album it was from, and if it was a single._

```
song_title
artist_name
year
album_name
single
```

Translate this information into CREATE TABLE statement with the correct arguments 

Reference info: https://www.postgresql.org/docs/11/datatype.html

In [27]:
## TO-DO: Finish writing the CREATE TABLE statement with the correct arguments
try: 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS songs 
    (
        song_title VARCHAR,
        artist_name VARCHAR,
        year VARCHAR,
        album_name VARCHAR,
        single VARCHAR
    )
    """)
except psycopg2.Error as e:
    print("Error: Could not create table")
    print(e)

### TO-DO: Insert the following two rows in the table
`First Row:  "Across The Universe", "The Beatles", "1970", "False", "Let It Be"`

`Second Row: "The Beatles", "Think For Yourself", "False", "1965", "Rubber Soul"`

In [28]:
## TO-DO: Finish the INSERT INTO statement with the correct arguments
try: 
    cur.execute(
    """
    INSERT INTO songs (song_title, artist_name, year, album_name, single) \
    VALUES (%s, %s, %s, %s, %s)
    """ , ("Across The Universe", "The Beatles", "1970", "False", "Let It Be")
    )
except psycopg2.Error as e:
    print("Error: Could not create a row")
    print(e)
                
try: 
    cur.execute(
    """
    INSERT INTO songs (song_title, artist_name, year, album_name, single) \
    VALUES (%s, %s, %s, %s, %s)
    """ , ("The Beatles", "Think For Yourself", "False", "1965", "Rubber Soul")
    )
except psycopg2.Error as e:
    print("Error: Could not create a row")
    print(e)

### TO-DO: Validate your data was inserted into the table. 


In [29]:
## TO-DO: Finish the SELECT * Statement 
try: 
    cur.execute(
    """
        SELECT * FROM songs
    """)
except psycopg2.Error as e:
    print(e)

for crow in cur.fetchall():
    print(crow)

('Across The Universe', 'The Beatles', '1970', 'False', 'Let It Be')
('The Beatles', 'Think For Yourself', 'False', '1965', 'Rubber Soul')


### And finally close your cursor and connection. 

In [30]:
cur.close()
conn.close()

<h2><span style='color:blue'>Remove Environment</span></h2>

In [44]:
# Clears proxy
pids_kubectl_proxy = !ps -ef|fgrep 'kubectl port-forward'|fgrep $CHART_INSTANCE_NAME|cut -d ' ' -f4
!kill -9 {pids_kubectl_proxy[0]}

In [32]:
# Removes chart instances
!helm uninstall {CHART_INSTANCE_NAME}

release "dend-l1e1" uninstalled


In [33]:
# Removes persistent Volume
!kubectl get pvc|fgrep {CHART_INSTANCE_NAME}|cut -d ' '  -f1| xargs -t kubectl delete pvc

kubectl delete pvc data-dend-l1e1-postgresql-0
persistentvolumeclaim "data-dend-l1e1-postgresql-0" deleted
